# User Data and Repository

In [ ]:
user = "delt550999"
# Where are the configuration files relative to git repo path
conf_path = "delft3d-fm/autosubmit/conf/"
# Main platform of the experiment
hpcarch = "marenostrum5"
# A description of this experiment
exp_description = "Delft3D-FM"
project = "ehpc69"
host = "glogin1.bsc.es"
queue = "gp_ehpc"

#### Settings to use a different user or repository.

In [ ]:
!ssh-copy-id f"{user}@{host}"

In [ ]:
!ssh f"{user}@{host}"

In [ ]:
#BSC connection checker
scratch_dir = "/gpfs/scratch"
cmd = f"{user}@{host} echo Connected"
!ssh -o StrictHostKeyChecking=accept-new {cmd} 

# Clone Test Case

In [ ]:
# Repo
repo = "https://github.com/Deltares-research/EditoServices"
# Branch 
branch = "dflowfm-singularity" 
! git clone --branch $branch $repo

## Copy Test Case to cluster 

In [ ]:
!scp -r EditoServices/delft3d-fm/test-case $user@$host:from-edito/

In [ ]:
%%capture output --no-stderr
!autosubmit expid -min -repo {repo} -H {hpcarch} -b {branch} -conf {conf_path} -d "{exp_description}"
# -min: 
  # Generates a minimal configuration file that replaces the one shown in previous examples
  # Mainly used to load versioned configuration files, and the to-go configuration in AS 4+
# -b: Specifies the git branch
# -conf : Specifies the path to the versioned configuration files. The root folder is the git repository root.

In [ ]:
# Show expid output
output.show()
#Get Expid directly from this output for the other commands
expid_command_output = output.stdout
expid = expid_command_output.split("\r\n")[-2].split(" ")[1]
#If you want to use another expid, comment the previous lines and uncomment this one
expid = "a001"
%store expid

In [ ]:
# FileStructure
!ls /home/onyxia/autosubmit/{expid}
# Mount it in the file explorer
!ln -s /home/onyxia/autosubmit/{expid} . 2> /dev/null # in case folder is already mounted
# The exp_root in this example is the expid folder.
# We have the expid in our current_directory as a symbolic link
exp_root = expid
%store exp_root

In [ ]:
! echo $expid

In [ ]:
# This command downloads the git_repo and, by default, puts it inside the folder {exp_root}/proj/git_project
!autosubmit refresh {expid} -v

In [ ]:
# Autosubmit refresh uses the content inside this file, to know where are the rest of the config files
!cat {expid}/conf/minimal.yml

In [ ]:
!ls /home/onyxia/autosubmit/$expid

In [ ]:
# We can find the rest of the config in the DEFAULT.CUSTOM_CONFIG path
# %PROJDIR% is an AS_placeholder that points to {exp_root}/%PROJECT.PROJECT_DESTINATION%
!ls {exp_root}/proj/git_project/{conf_path}

In [ ]:
#In this repo, we can find the templates under %PROJDIR%/templates
!ls {exp_root}/proj/git_project/templates

In [ ]:
# This step is slightly different than in the other notebooks.
# In this expid, the platform.yml is already there with the definition of the platforms. 
#!cat {exp_root}/proj/git_project/as_conf/platforms.yml
# We will overwrite only what we want to change from that file.
# This means that instead of filling the whole platforms.yml we will fill only the necessary.
platforms_file = f"""PLATFORMS:
  MARENOSTRUM5:
    TYPE: slurm
    HOST: {host}
    PROJECT: {project}
    USER: {user}
    QUEUE: {queue}
    SCRATCH_DIR: /gpfs/scratch
    ADD_PROJECT_TO_HOST: False
    MAX_WALLCLOCK: '48:00'
"""
# And store it in our home directory.
import os 
with open(f"{os.environ['HOME']}/platforms.yml", "w") as fd:
    fd.write(platforms_file)

# For Autosubmit to acknowledge this file, we must modify the {exp_root}/conf/minimal.yml %DEFAULT.CUSTOM_CONFIG% key and add a POST key. 
# You can also use the file_explorer
minimal=f"""CONFIG:
  # Current version of Autosubmit.
  AUTOSUBMIT_VERSION: "4.1.10"
  # Total number of jobs in the workflow.
  TOTALJOBS: 20
  # Maximum number of jobs permitted in the waiting status.
  MAXWAITINGJOBS: 20
DEFAULT:
  # Job experiment ID.
  EXPID: "{expid}"
  # Default HPC platform name.
  HPCARCH: {hpcarch}
  #hint: use %PROJDIR% to point to the project folder (where the project is cloned)
  # Custom configuration location.
  CUSTOM_CONFIG: 
   PRE: "%PROJDIR%/{conf_path}"
   POST: "~/platforms.yml"
PROJECT:
  # Type of the project.
  PROJECT_TYPE: git
  # Folder to hold the project sources.
  PROJECT_DESTINATION: git_project
EXPERIMENT:
  DATELIST: "20000101"
  MEMBERS: "fc0"
  CHUNKSIZEUNIT: month
  CHUNKSIZE: "4"
  NUMCHUNKS: "2"
  CHUNKINI: ''
  CALENDAR: standard
GIT:
  PROJECT_ORIGIN: "{repo}"
  PROJECT_BRANCH: "{branch}"
  PROJECT_COMMIT: ''
  PROJECT_SUBMODULES: ''
  FETCH_SINGLE_BRANCH: true
"""
# And store it in the conf path
with open(f"{expid}/conf/minimal.yml", "w") as fd:
    fd.write(minimal)

In [ ]:
%%capture output --no-stderr
!autosubmit create {expid} -v

In [ ]:
# Capture the file_path to the plot
create_command_output = output.stdout
parsed_by_lines = create_command_output.split("\r\n")
for line in parsed_by_lines:
    if "Plot created".casefold() in line.casefold():
        filepath = line.split(" ")[-1].split(".")[0] +".pdf"
print(filepath) 
# Then plot it using Jupyter Python.
from IPython.display import IFrame
from pathlib import Path
IFrame(f"{expid}/plot/{Path(filepath).name}", width=600, height=300)

In [ ]:
!autosubmit run {expid} 

> NOTE: In the next sessions you will learn to use the Autosubmit GUI to monitor your experiments. Here we will use the Autosubmit command-line interface for that.

In [ ]:
%%capture monitor_output --no-stderr
!autosubmit monitor {expid}

# Copy Whole model Back from Cluster (with results)

In [ ]:
!scp -r $user@$host:from-edito/dflowfm/ results/

In [ ]:
# Capture the file_path to the plot
monitor_command_output = monitor_output.stdout
parsed_by_lines = monitor_command_output.split("\r\n")
for line in parsed_by_lines:
    if "Plot created".casefold() in line.casefold():
        filepath = line.split(" ")[-1].split(".")[0] +".pdf"
print(filepath) 
# Then plot it using Jupyter Python.
from IPython.display import IFrame
from pathlib import Path
IFrame(f"{expid}/plot/{Path(filepath).name}", width=600, height=300)

In [ ]:
#Aslogs are located inside
!ls -hart {exp_root}/tmp/ASLOGS

In [ ]:
#template logs are downloaded from the remote platform and stored inside
!ls -hart {exp_root}/tmp/LOG_{expid}

In [ ]:
# Workflow ran inside
cmd = f"{user}@{host} ls -l {scratch_dir}/{project}/{user}/{expid}"
!ssh -o StrictHostKeyChecking=accept-new {cmd} 

In [ ]:
# TODO: we need to export this for now due to an issue in the service configuration
import os
%env MC_HOST_s3=https://{os.environ["AWS_ACCESS_KEY_ID"]}:{os.environ["AWS_SECRET_ACCESS_KEY"]}:{os.environ["AWS_SESSION_TOKEN"]}@{os.environ["AWS_S3_ENDPOINT"]}
# Persist the work
# If you are launching from your own project, use this
!mc cp --recursive /home/onyxia/autosubmit s3/oidc-{edito_user}
# If you are launching from project-modellab project, use this
# !mc cp --recursive /home/onyxia/autosubmit s3/project-modellab/{edito_user}